In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import re

def analyze_expansion_data(file_path):
    # --- 1. 读取数据 ---
    print("正在读取数据...")
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"错误：找不到文件 {file_path}")
        return

    # --- 列名清洗 ---
    df.columns = df.columns.astype(str).str.replace('\n', '').str.strip()
    
    # --- 2. 数据清洗与预处理 ---
    print("正在清洗数据...")
    
    text_cols = ['133行业名称', '市场化属性分类名称', '业扩业务类型', '用户名称', '用户编号', '用电类别名称']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    numeric_cols = [
        '原有合同容量', '申请合同容量', '原有运行容量', '申请运行容量', '运行容量', '合同容量',
        '业扩变更前预计2月份日均电量', '业扩变更后预计2月份日均电量', '业扩变更后预计3月份日均电量',
        '2026年2月第一周电量', '2025年12月电量'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        else:
            df[col] = 0 

    # 万能日期解析
    date_col = '预计送（停）电日期'
    if date_col not in df.columns:
        potential_cols = [c for c in df.columns if '送' in c and '日期' in c]
        if potential_cols:
            date_col = potential_cols[0]
        else:
            print("错误：无法找到送电日期列。")
            return

    def parse_flexible_date(val):
        if pd.isna(val): return pd.NaT
        s = str(val).strip()
        if s.isdigit() and len(s) == 8:
            return pd.to_datetime(s, format='%Y%m%d', errors='coerce')
        if s.replace('.', '', 1).isdigit() and float(s) < 100000:
             return pd.Timestamp('1899-12-30') + pd.Timedelta(days=float(s))
        clean_s = re.sub(r'[./年月]', '-', s).replace('日', '')
        try:
            return pd.to_datetime(clean_s, errors='coerce')
        except:
            return pd.NaT

    df['送电日期_dt'] = df[date_col].apply(parse_flexible_date)

    # --- 3. 核心逻辑：计算“有效变动容量” ---
    print("正在计算有效变动容量...")
    
    def get_effective_capacity(row):
        b_type = row['业扩业务类型']
        if '新装' in b_type:
            return row['合同容量']
        elif '增容' in b_type:
            return row['申请合同容量'] if row['申请合同容量'] > 0 else row['申请运行容量']
        elif '减容' in b_type:
            if row['申请运行容量'] > 0:
                return row['申请运行容量']
            else:
                diff = row['原有运行容量'] - row['运行容量']
                return diff if diff > 0 else 0
        return 0

    df['有效变动容量'] = df.apply(get_effective_capacity, axis=1)

    # --- 【新增】物理极限校验模块 ---
    print("正在执行物理极限校验 (基于容量判断)...")

    def correct_by_physics(row):
        # 获取需要检查的三个日均值
        val_feb_pre = row['业扩变更前预计2月份日均电量']
        val_feb_post = row['业扩变更后预计2月份日均电量']
        val_mar_post = row['业扩变更后预计3月份日均电量']
        
        # 获取容量 (如果是减容，这里取的是减少的量，用来估算减少的电量是否合理；
        # 但对于变更后的日均，应该用剩余的总容量来判断。
        # 这里为了简化和通用，我们使用该用户最大的那个容量值作为物理上限的基准)
        # 比如：运行容量、合同容量、申请容量里的最大值
        max_cap = max(row['运行容量'], row['合同容量'], row['申请运行容量'], row['申请合同容量'], row['有效变动容量'])
        
        if max_cap == 0:
            return val_feb_pre, val_feb_post, val_mar_post
            
        # 理论最大日电量 = 容量 * 24小时
        # 给予 1.5 倍的容错空间 (考虑到超负荷或功率因数等极端情况，或者填报的容量单位误差)
        limit_daily = max_cap * 24 * 1.5
        
        # 定义修正函数
        def fix_val(val, limit):
            if val > limit:
                # 尝试判断是否为月度总量 (除以30后在合理范围内)
                if (val / 30) < limit:
                    return val / 31.0 # 粗略按31天算，或者按当月实际天数
                # 尝试判断是否为周度总量
                elif (val / 7) < limit:
                    return val / 7.0
            return val

        # 修正三个值
        val_feb_pre = fix_val(val_feb_pre, limit_daily)
        val_feb_post = fix_val(val_feb_post, limit_daily)
        val_mar_post = fix_val(val_mar_post, limit_daily)
        
        return val_feb_pre, val_feb_post, val_mar_post

    # 应用物理修正
    corrected_physics = df.apply(correct_by_physics, axis=1, result_type='expand')
    df['业扩变更前预计2月份日均电量'] = corrected_physics[0]
    df['业扩变更后预计2月份日均电量'] = corrected_physics[1]
    df['业扩变更后预计3月份日均电量'] = corrected_physics[2]

    # --- 【原有】历史数据比对纠错模块 ---
    print("正在执行历史数据比对纠错...")
    
    def correct_daily_usage_history(row):
        val_feb_pre = row['业扩变更前预计2月份日均电量']
        val_feb_post = row['业扩变更后预计2月份日均电量']
        val_mar_post = row['业扩变更后预计3月份日均电量']
        
        ref_feb_week = row['2026年2月第一周电量'] 
        ref_dec_month = row['2025年12月电量']     
        
        # 修正 2月 (对比周总量)
        if ref_feb_week > 10 and val_feb_pre > 10:
            if 0.5 <= (val_feb_pre / ref_feb_week) <= 2.0:
                val_feb_pre = val_feb_pre / 7.0
        
        if ref_feb_week > 10 and val_feb_post > 10:
            if 0.5 <= (val_feb_post / ref_feb_week) <= 5.0: 
                val_feb_post = val_feb_post / 7.0

        # 修正 3月 (对比月总量)
        if ref_dec_month > 10 and val_mar_post > 10:
            if 0.5 <= (val_mar_post / ref_dec_month) <= 5.0:
                val_mar_post = val_mar_post / 31.0
                
        return val_feb_pre, val_feb_post, val_mar_post

    # 应用历史比对修正
    corrected_history = df.apply(correct_daily_usage_history, axis=1, result_type='expand')
    df['业扩变更前预计2月份日均电量'] = corrected_history[0]
    df['业扩变更后预计2月份日均电量'] = corrected_history[1]
    df['业扩变更后预计3月份日均电量'] = corrected_history[2]

    # --- 4. 核心逻辑：计算电量增量 ---
    print("正在计算电量增量...")

    df['日均差值_2月'] = df['业扩变更后预计2月份日均电量'] - df['业扩变更前预计2月份日均电量']
    df['日均差值_3月'] = df['业扩变更后预计3月份日均电量'] - df['业扩变更后预计2月份日均电量']

    weeks_config = [
        ('2月第一周', pd.Timestamp('2026-02-02'), pd.Timestamp('2026-02-08')),
        ('2月第二周', pd.Timestamp('2026-02-09'), pd.Timestamp('2026-02-15')),
        ('2月第三周', pd.Timestamp('2026-02-16'), pd.Timestamp('2026-02-22')),
        ('2月第四周', pd.Timestamp('2026-02-23'), pd.Timestamp('2026-02-28'))
    ]

    def calculate_increment_and_days(row, start_date, end_date, daily_inc):
        if pd.isna(row['送电日期_dt']): return 0, 0
        effect_date = row['送电日期_dt']
        if effect_date > end_date: return 0, 0
        actual_start = max(start_date, effect_date)
        if actual_start > end_date: return 0, 0
        days = (end_date - actual_start).days + 1
        return days * daily_inc, days

    # 4.1 周增量
    for week_name, w_start, w_end in weeks_config:
        df[f'{week_name}增量'] = df.apply(
            lambda row: calculate_increment_and_days(row, w_start, w_end, row['日均差值_2月'])[0], axis=1
        )

    # 4.2 2月合计
    feb_start = pd.Timestamp('2026-02-01')
    feb_end = pd.Timestamp('2026-02-28')
    temp_res_feb = df.apply(
        lambda row: calculate_increment_and_days(row, feb_start, feb_end, row['日均差值_2月']), axis=1
    )
    df['2月合计增量'] = temp_res_feb.apply(lambda x: x[0])
    df['2月有效天数'] = temp_res_feb.apply(lambda x: x[1])

    # 4.3 3月合计
    mar_start = pd.Timestamp('2026-03-01')
    mar_end = pd.Timestamp('2026-03-31')
    df['3月总增量'] = df.apply(
        lambda row: calculate_increment_and_days(row, mar_start, mar_end, row['日均差值_3月'])[0], axis=1
    )

    # --- 5. 聚合分析 ---
    print("正在生成分析报表...")

    agg_funcs_cap = {'有效变动容量': 'sum', '用户编号': 'count'}
    agg_funcs_power = {
        '2月第一周增量': 'sum', '2月第二周增量': 'sum', '2月第三周增量': 'sum', 
        '2月第四周增量': 'sum', '2月合计增量': 'sum', '3月总增量': 'sum'
    }

    df_industry_cap = df.groupby(['133行业名称', '业扩业务类型']).agg(agg_funcs_cap).reset_index()
    df_industry_cap.rename(columns={'用户编号': '户数', '有效变动容量': '容量(kVA)'}, inplace=True)
    
    df_industry_power = df.groupby(['133行业名称']).agg(agg_funcs_power).reset_index()
    df_industry_power = df_industry_power.sort_values(by='2月合计增量', ascending=False)

    df_market_cap = df.groupby(['市场化属性分类名称', '业扩业务类型']).agg(agg_funcs_cap).reset_index()
    df_market_cap.rename(columns={'用户编号': '户数', '有效变动容量': '容量(kVA)'}, inplace=True)
    df_market_power = df.groupby(['市场化属性分类名称']).agg(agg_funcs_power).reset_index()
    
    df_usage_type_cap = df.groupby(['用电类别名称', '业扩业务类型']).agg(agg_funcs_cap).reset_index()
    df_usage_type_cap.rename(columns={'用户编号': '户数', '有效变动容量': '容量(kVA)'}, inplace=True)
    df_usage_type_power = df.groupby(['用电类别名称']).agg(agg_funcs_power).reset_index()
    df_usage_type_power = df_usage_type_power.sort_values(by='2月合计增量', ascending=False)

    # TOP 用户
    df['2月增量绝对值'] = df['2月合计增量'].abs()
    df_top_users_feb = df.sort_values(by='2月增量绝对值', ascending=False).head(50)
    
    df['3月增量绝对值'] = df['3月总增量'].abs()
    df_top_users_mar = df.sort_values(by='3月增量绝对值', ascending=False).head(50)

    top_user_cols = [
        '地市', '用户名称', '133行业名称', '用电类别名称', '业扩业务类型', '有效变动容量', '预计送（停）电日期', 
        '日均差值_2月', '日均差值_3月', '2月合计增量', '3月总增量'
    ]
    existing_cols = [c for c in top_user_cols if c in df.columns]

    # --- 6. 导出结果 ---
    output_path = os.path.join(os.path.dirname(file_path), '业扩摸排数据_分析结果.xlsx')
    print(f"正在保存结果至: {output_path}")
    
    with pd.ExcelWriter(output_path) as writer:
        df_industry_cap.to_excel(writer, sheet_name='行业_容量及户数', index=False)
        df_market_cap.to_excel(writer, sheet_name='市场化_容量及户数', index=False)
        df_industry_power.to_excel(writer, sheet_name='行业_电量增量预测', index=False)
        df_market_power.to_excel(writer, sheet_name='市场化_电量增量预测', index=False)
        df_usage_type_cap.to_excel(writer, sheet_name='用电类别_容量及户数', index=False)
        df_usage_type_power.to_excel(writer, sheet_name='用电类别_电量增量预测', index=False)
        
        df_top_users_feb[existing_cols].to_excel(writer, sheet_name='TOP50_2月增量用户', index=False)
        df_top_users_mar[existing_cols].to_excel(writer, sheet_name='TOP50_3月增量用户', index=False)
        
        detail_cols = [
            '用户名称', '用电类别名称', '业扩业务类型', '送电日期_dt', '有效变动容量',
            '业扩变更前预计2月份日均电量', '业扩变更后预计2月份日均电量', '业扩变更后预计3月份日均电量',
            '日均差值_2月', '日均差值_3月', '2月合计增量', '3月总增量'
        ]
        df[detail_cols].to_excel(writer, sheet_name='计算明细核对', index=False)

    print("分析完成！")

if __name__ == '__main__':
    file_path = r"E:\A智网\业扩分析\26年2月分析\业扩摸排数据.xlsx"
    analyze_expansion_data(file_path)

正在读取数据...
正在清洗数据...
正在计算有效变动容量...
正在执行物理极限校验 (基于容量判断)...
正在执行历史数据比对纠错...
正在计算电量增量...
正在生成分析报表...
正在保存结果至: E:\A智网\业扩分析\26年2月分析\业扩摸排数据_分析结果.xlsx
分析完成！
